In [1]:
from nltk.tag import StanfordNERTagger
from nltk.tokenize import word_tokenize
import numpy as np
import pandas as pd
import tools as t
import pickle
import matplotlib.pyplot as plt
import preprocessor as p
from collections import defaultdict
import re
import json
import StringIO
from os import listdir
from os.path import isfile, join,isdir
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (10.0, 8.0)
%matplotlib inline
%load_ext autoreload 
%autoreload 2

In [2]:
def read_fileIn_array(filename):
    news=np.array(pd.read_csv(filename))
    return news

In [3]:
filename='./geo.csv'

In [15]:
def get_entities(news):
    classifier = '/usr/share/stanford-ner/classifiers/english.all.3class.distsim.crf.ser.gz'
    jar = '/usr/share/stanford-ner/stanford-ner.jar'
    st = StanfordNERTagger(classifier,jar)
    news_entities=[]
    for i in range(0,len(news)):
        temp_news=news[i][2]
        if type(temp_news)==str:        
            temp_news=temp_news.decode("utf8")
            sentence = word_tokenize(temp_news)
            tagged=st.tag(sentence)
            news_entities.append(np.array(tagged))
    it_entities=list()
    for entities in news_entities:
        tagged=np.array(entities)
        tagged=tagged[tagged[:,1]!='O']
        tagged=tagged[tagged[:,1]=='PERSON'][:,0]
        for tag in tagged:
            it_entities.append(tag.encode("utf-8"))
    return it_entities     

In [ ]:
news=read_fileIn_array(filename)


In [ ]:
it_entities=get_entities(news)

In [ ]:
print len(it_entities)

In [ ]:
temp=np.array(it_entities)

In [ ]:
print len(it_entities)

In [ ]:
fname="tweets/entities.csv"
df=pd.DataFrame(it_entities)
df.to_csv(fname)

In [4]:
df=pd.read_csv("entities.csv")

In [5]:
df=df['0']

In [6]:
arr=np.array(df)

In [7]:
temp_dict=dict()

In [1]:
import tweepy
from tweepy import OAuthHandler
import json
import datetime as dt
import time
import os
import sys


def load_api():

    consumer_key = 'duhAXGFbpB72qajYFM2tnxJHb'
    consumer_secret = '0i3Pilasyr0zLkSgEe2oTaF0URqHZCLF3t6uZ3gFT7IqDFZGz1'
    access_token = '897788995668762624-rmsX4hvUmpCXDhCtKGgVB9Cs3JHnpve'
    access_secret = 'kNH2m8KxQEYJWiban0yXNNhHEszfrTU5p5UjDXtXiLtMv'
    auth = OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_secret)
    # load the twitter API via tweepy
    return tweepy.API(auth)

    
def tweet_search(api, query, max_tweets, max_id, since_id, geocode):

    searched_tweets = []
    while len(searched_tweets) < max_tweets:
        remaining_tweets = max_tweets - len(searched_tweets)
        try:
            new_tweets = api.search(q=query, count=remaining_tweets,
                                    since_id=str(since_id),       max_id=str(max_id-1))
#                                    geocode=geocode)
            print('found',len(new_tweets),'tweets')
            if not new_tweets:
                print('no tweets found')
                break
            searched_tweets.extend(new_tweets)
            max_id = new_tweets[-1].id
        except tweepy.TweepError:
            print('exception raised, waiting 15 minutes')
            print('(until:', dt.datetime.now()+dt.timedelta(minutes=15), ')')
            time.sleep(15*60)
            break # stop the loop
    return searched_tweets, max_id


def get_tweet_id(api, date='', days_ago=9, query='a'):

    if date:
        # return an ID from the start of the given day
        td = date + dt.timedelta(days=1)
        tweet_date = '{0}-{1:0>2}-{2:0>2}'.format(td.year, td.month, td.day)
        tweet = api.search(q=query, count=1, until=tweet_date)
    else:
        # return an ID from __ days ago
        td = dt.datetime.now() - dt.timedelta(days=days_ago)
        tweet_date = '{0}-{1:0>2}-{2:0>2}'.format(td.year, td.month, td.day)
        # get list of up to 10 tweets
        tweet = api.search(q=query, count=10, until=tweet_date)
        print('search limit (start/stop):',tweet[0].created_at)
        # return the id of the first tweet in the list
        return tweet[0].id


def write_tweets(tweets, filename):
    ''' Function that appends tweets to a file. '''

    with open(filename, 'a+') as f:
        for tweet in tweets:
            json.dump(tweet._json, f)
            f.write('\n')


def main():
    ''' This is a script that continuously searches for tweets
        that were created over a given number of days. The search
        dates and search phrase can be changed below. '''



    ''' search variables: '''
    search_phrases = arr
    time_limit = 1.5                           # runtime limit in hours
    max_tweets = 100                           # number of tweets per search (will be
                                               # iterated over) - maximum is 100
    min_days_old=0
    max_days_old =  7                        # search limits e.g., from 7 to 8
                                               # gives current weekday from last week,
                                               # min_days_old=0 will search from right now
    USA = '33.69458,73.06437,500km'       

    # loop over search items,
    # creating a new file for each
    searched_dict=dict()
    for search_phrase in search_phrases:
        
        if search_phrase not in searched_dict:
            searched_dict[search_phrase]=0
            print('Search phrase =', search_phrase)

            ''' other variables '''
            name = search_phrase.split()[0]
            json_file_root = "raw_tweets/"+name + '/'  + name
            temp_file_root="raw_tweets/"+name 
         
            if not isdir(temp_file_root):
                os.makedirs(os.path.dirname(json_file_root))
                
            read_IDs = False

            # open a file in which to store the tweets
            if max_days_old - min_days_old == 1:
                d = dt.datetime.now() - dt.timedelta(days=min_days_old)
                day = '{0}-{1:0>2}-{2:0>2}'.format(d.year, d.month, d.day)
            else:
                d1 = dt.datetime.now() - dt.timedelta(days=max_days_old-1)
                d2 = dt.datetime.now() - dt.timedelta(days=min_days_old)
                day = '{0}-{1:0>2}-{2:0>2}_to_{3}-{4:0>2}-{5:0>2}'.format(
                      d1.year, d1.month, d1.day, d2.year, d2.month, d2.day)
            
            json_file = json_file_root + '_' + day + '.json'
            
            if os.path.isfile(json_file):
                print('Appending tweets to file named: ',json_file)
                read_IDs = True

            # authorize and load the twitter API
            api = load_api()

            # set the 'starting point' ID for tweet collection
            if read_IDs:
                # open the json file and get the latest tweet ID
                with open(json_file, 'a+') as f:
                    lines = f.readlines()
                    max_id = json.loads(lines[-1])['id']
                    print('Searching from the bottom ID in file')
            else:
                # get the ID of a tweet that is min_days_old
                if min_days_old == 0:
                    max_id = -1
                else:
                    max_id = get_tweet_id(api, days_ago=(min_days_old-1))
            # set the smallest ID to search for
            since_id = get_tweet_id(api, days_ago=(max_days_old-1))
            print('max id (starting point) =', max_id)
            print('since id (ending point) =', since_id)



            ''' tweet gathering loop  '''
            start = dt.datetime.now()
            end = start + dt.timedelta(hours=time_limit)
            count, exitcount = 0, 0
            while dt.datetime.now() < end and count<5:
                count += 1
                print('count =',count)
                # collect tweets and update max_id
                tweets, max_id = tweet_search(api, search_phrase, max_tweets,
                                              max_id=max_id, since_id=since_id,
                                   geocode=USA )
                # write tweets to file in JSON format
                if tweets:
                    write_tweets(tweets, json_file)
                    exitcount = 0
                else:
                    exitcount +=1
                    if exitcount == 3:
                        if search_phrase == search_phrases[-1]:
                            sys.exit('Maximum number of empty tweet strings reached - exiting')
                        else:
                            print('Maximum number of empty tweet strings reached - breaking')
                            break


if __name__ == "__main__":
    main()

In [9]:
def tweet_cleaner(filenames,name):
    tweets=list()
    for filename in filenames:
        if isfile(filename):
            with open(filename) as fr:
                data = fr.read()
                json_list=data.split('\n')
                for i in range(len(json_list)-1):
                    obj=json.loads(json_list[i])
                    text=obj["text"]
                    tweets.append(text.encode("utf-8"))
    fname="tweets/"+name+".csv"
    df=pd.DataFrame(tweets)
    df.to_csv(fname)

In [10]:
mypaths=arr
for name in mypaths:
    if name not in temp_dict:
        temp_dict[name]=0
    path="raw_tweets/"+name
    if isdir(path):
        onlyfiles = [join(path, f) for f in listdir(path) if isfile(join(path, f))]
        tweet_cleaner(onlyfiles,name)

In [11]:
def preprocess(s, lowercase=False):
    s = str(s[1])
    txt=p.clean(s)
    txt = re.sub('[\n]',' ',txt)
    txt = re.sub('[:[]-_]',' ',txt)
    return word_tokenize(txt)


In [12]:
def tweet_classifier(filename):
    tweets=read_fileIn_array(filename)
    for i in range(0,10):
        print preprocess(tweets[i])
        

In [13]:
fname="tweets/Khawaja.csv"
tweet_classifier(fname)

[u':', u'This', u'man', u'Khawaja', u'Shams', u'is', u'a', u'Senior', u'Lawyer', u'who', u'had', u'a', u'run', u'in', u'with', u'the', u'Police', u'.', u'The', u'fact', u'that', u'whether', u'he', u'was', u'racing', u'or', u'not', u'is\u2026']
[u':', u'This', u'man', u'Khawaja', u'Shams', u'is', u'a', u'Senior', u'Lawyer', u'who', u'had', u'a', u'run', u'in', u'with', u'the', u'Police', u'.', u'The', u'fact', u'that', u'whether', u'he', u'was', u'racing', u'or', u'not', u'is\u2026']
[u'I', u'guess', u'all', u'they', u'can', u'really', u'do', u'is', u'take', u'it', u'into', u'open', u'fields', u'and', u'blow', u'it', u'up', u'!', u'!']
[u':', u'This', u'man', u'Khawaja', u'Shams', u'is', u'a', u'Senior', u'Lawyer', u'who', u'had', u'a', u'run', u'in', u'with', u'the', u'Police', u'.', u'The', u'fact', u'that', u'whether', u'he', u'was', u'racing', u'or', u'not', u'is\u2026']
[u':', u'This', u'man', u'Khawaja', u'Shams', u'is', u'a', u'Senior', u'Lawyer', u'who', u'had', u'a', u'run', u'